In [1]:
#import libraries
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy.stats import norm
from scipy.stats import zscore
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [40]:
# load train set and test set
df = pd.read_csv("https://raw.githubusercontent.com/MathewJohn1986/Team-8-Predict/main/df-train_set.csv")


In [41]:
test = pd.read_csv('https://raw.githubusercontent.com/MathewJohn1986/Team-8-Predict/main/df-test_set.csv')

In [50]:
#filtering out the Apple golden delicious
apples_df_filter = df[df['Commodities']== 'APPLE GOLDEN DELICIOUS']
apples_df = apples_df_filter.copy()
apples_df.head()

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,CAPE,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
7,CAPE,JG110,2M,11.0,APPLE GOLDEN DELICIOUS,2020-04-14,50.0,50.0,16000.0,320,3520.0,0,4.55
24,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,APPLE GOLDEN DELICIOUS,2020-04-16,55.0,55.0,990.0,18,162.0,1506,6.11
40,CAPE,M4183,1S,18.3,APPLE GOLDEN DELICIOUS,2020-05-04,80.0,120.0,32020.0,388,7100.4,443,4.51
69,EASTERN CAPE,IA400,1S,400.0,APPLE GOLDEN DELICIOUS,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2,4.50


In [57]:
test.head()

,Index,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
0,1,W.CAPE-BERGRIVER ETC,EC120,1M,2.484907,APPLE GOLDEN DELICIOUS,2020-07-09,4.852030,4.912655,8.518792,3.637586,6.122493,0.000000
1,2,W.CAPE-BERGRIVER ETC,M4183,1X,2.906901,APPLE GOLDEN DELICIOUS,2020-01-20,5.393628,5.393628,7.473069,2.079442,4.986343,0.693147
2,3,W.CAPE-BERGRIVER ETC,EC120,1S,2.484907,APPLE GOLDEN DELICIOUS,2020-08-19,4.787492,4.787492,6.579251,1.791759,4.276666,3.806662
3,4,W.CAPE-BERGRIVER ETC,M4183,1M,2.906901,APPLE GOLDEN DELICIOUS,2020-05-06,5.075174,5.075174,5.075174,0.000000,2.906901,2.079442
4,5,W.CAPE-BERGRIVER ETC,M4183,1L,2.906901,APPLE GOLDEN DELICIOUS,2020-05-04,4.941642,5.075174,9.556763,4.605170,7.512071,2.944439


In [45]:
apples_df.skew()

Weight_Kg           3.328254
Low_Price           4.222672
High_Price          3.828819
Sales_Total         4.250317
Total_Qty_Sold      4.445169
Total_Kg_Sold       5.529586
Stock_On_Hand       3.391815
avg_price_per_kg    0.082452
dtype: float64

In [46]:
test.skew()

Index             0.000000
Weight_Kg         3.941111
Low_Price         4.585123
High_Price        4.326922
Sales_Total       3.839600
Total_Qty_Sold    3.710551
Total_Kg_Sold     4.270965
Stock_On_Hand     3.197405
dtype: float64

In [52]:
cats = ['Weight_Kg', 'Low_Price', 'High_Price', 'Sales_Total', 'Total_Qty_Sold', 'Total_Kg_Sold', 'Stock_On_Hand']
for a in cats:
    apples_df[a] = apples_df[a].map(lambda i: np.log(i) if i > 0 else 0)
    test[a] = test[a].map(lambda i: np.log(i) if i > 0 else 0)

In [53]:
test.skew()

Index             0.000000
Weight_Kg         2.995465
Low_Price         1.631810
High_Price        1.443425
Sales_Total      -0.397272
Total_Qty_Sold   -0.326266
Total_Kg_Sold    -0.376128
Stock_On_Hand    -0.648157
dtype: float64

In [54]:
apples_df.skew()

Weight_Kg           2.624779
Low_Price           1.141070
High_Price          1.089270
Sales_Total        -0.482850
Total_Qty_Sold     -0.362500
Total_Kg_Sold      -0.411865
Stock_On_Hand      -0.511006
avg_price_per_kg    0.082452
dtype: float64

In [55]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

apples_df = apples_df[~((apples_df < (Q1 - 1.5 * IQR)) |(apples_df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [60]:
apples_df['Province'].unique()

array(['CAPE', 'W.CAPE-BERGRIVER ETC', 'EASTERN CAPE', 'NATAL',
       'WEST COAST', 'TRANSVAAL', 'ORANGE FREE STATE'], dtype=object)

####  Categorical variables

In [61]:
codes = {"3.0": 0, "6.3": 1, "9.0": 2, "11.0": 3, "12.0": 4, "12.5": 5, "14.0": 6, "18.3": 7, "400.0": 8}
apples_df['Weight_Kg']= apples_df['Weight_Kg'].map(codes)
test['Weight_Kg']= test['Weight_Kg'].map(codes)

In [63]:
codes = {"CAPE": 0, "W.CAPE-BERGRIVER ETC": 1, "EASTERN CAPE": 2, "NATAL": 3, "ORANGE FREE STATE": 4, "TRANSVAAL": 5, "WEST COAST": 6}
apples_df['Province']= apples_df['Province'].map(codes)
test['Province']= test['Province'].map(codes)

In [65]:
codes = {"M4183": 0, "JG110": 1, "JE090": 2, "IA400": 3, "EC120": 4, "AC030": 5, "M6125": 6, "EF120": 7, "DT063": 8, "M9125": 9, "EG140": 10}
apples_df['Container']= apples_df['Container'].map(codes)
test['Container']= test['Container'].map(codes)

In [66]:
cocodes = {"1L": 0, "2M": 1, "2S": 2, "1S": 3, "1M": 4, "1X": 5, "2L": 6, "2U": 7, "2X": 8, "1U": 9}
apples_df['Size_Grade']= apples_df['Size_Grade'].map(codes)
test['Size_Grade']= test['Size_Grade'].map(codes)

In [67]:
apples_df.drop(['Commodities','Date'],axis=1,inplace=True)
test.drop(['Commodities','Date'],axis=1,inplace=True)

In [69]:
apples_df.isnull().sum()

Province               0
Container              0
Size_Grade          1951
Weight_Kg           1951
Low_Price              0
High_Price             0
Sales_Total            0
Total_Qty_Sold         0
Total_Kg_Sold          0
Stock_On_Hand          0
avg_price_per_kg       0
dtype: int64

In [70]:
apples_df.drop(['Size_Grade','Weight_Kg'],axis=1,inplace=True)
apples_df.isnull().sum()

Province            0
Container           0
Low_Price           0
High_Price          0
Sales_Total         0
Total_Qty_Sold      0
Total_Kg_Sold       0
Stock_On_Hand       0
avg_price_per_kg    0
dtype: int64

In [74]:
apples_df.head()

,Province,Container,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,0,0,5.010635,5.135798,10.853406,5.805135,8.712036,6.711740,8.51
7,0,1,3.912023,3.912023,9.680344,5.768321,8.166216,0.000000,4.55
24,1,2,4.007333,4.007333,6.897705,2.890372,5.087596,7.317212,6.11
40,0,0,4.382027,4.787492,10.374116,5.961005,8.867906,6.093570,4.51
69,2,3,7.495542,7.495542,7.495542,0.000000,5.991465,0.693147,4.50


In [71]:
test.head()

,Index,Province,Container,Size_Grade,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
0,1,1,4,NaN,NaN,4.852030,4.912655,8.518792,3.637586,6.122493,0.000000
1,2,1,0,NaN,NaN,5.393628,5.393628,7.473069,2.079442,4.986343,0.693147
2,3,1,4,NaN,NaN,4.787492,4.787492,6.579251,1.791759,4.276666,3.806662
3,4,1,0,NaN,NaN,5.075174,5.075174,5.075174,0.000000,2.906901,2.079442
4,5,1,0,NaN,NaN,4.941642,5.075174,9.556763,4.605170,7.512071,2.944439


In [72]:
test.isnull().sum()

Index               0
Province            0
Container           0
Size_Grade        685
Weight_Kg         685
Low_Price           0
High_Price          0
Sales_Total         0
Total_Qty_Sold      0
Total_Kg_Sold       0
Stock_On_Hand       0
dtype: int64

In [73]:
test.drop(['Size_Grade','Weight_Kg'],axis=1,inplace=True)
test.isnull().sum()

Index             0
Province          0
Container         0
Low_Price         0
High_Price        0
Sales_Total       0
Total_Qty_Sold    0
Total_Kg_Sold     0
Stock_On_Hand     0
dtype: int64

In [75]:
apples_df.head()

,Province,Container,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,0,0,5.010635,5.135798,10.853406,5.805135,8.712036,6.711740,8.51
7,0,1,3.912023,3.912023,9.680344,5.768321,8.166216,0.000000,4.55
24,1,2,4.007333,4.007333,6.897705,2.890372,5.087596,7.317212,6.11
40,0,0,4.382027,4.787492,10.374116,5.961005,8.867906,6.093570,4.51
69,2,3,7.495542,7.495542,7.495542,0.000000,5.991465,0.693147,4.50


In [76]:
test.head()

,Index,Province,Container,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
0,1,1,4,4.852030,4.912655,8.518792,3.637586,6.122493,0.000000
1,2,1,0,5.393628,5.393628,7.473069,2.079442,4.986343,0.693147
2,3,1,4,4.787492,4.787492,6.579251,1.791759,4.276666,3.806662
3,4,1,0,5.075174,5.075174,5.075174,0.000000,2.906901,2.079442
4,5,1,0,4.941642,5.075174,9.556763,4.605170,7.512071,2.944439


### Modelling

In [77]:
X_train = apples_df.drop(['avg_price_per_kg'], axis=1)
Y_train = apples_df["avg_price_per_kg"]
X_test  = test.drop("Index", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((1951, 8), (1951,), (685, 8))

In [78]:
xgb = XGBRegressor()
xgb.fit(X_train,Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [79]:
y_pred = xgb.predict(X_test)
y_pred

array([11.009044 , 11.942808 , 10.043292 ,  8.694312 ,  7.9229555,
        4.6186833,  3.6957679,  7.4431124,  8.422279 ,  9.393129 ,
        5.494694 ,  5.6990128,  8.528781 ,  4.310257 ,  4.507008 ,
        6.0917335,  6.9610457,  4.507426 , 10.9204235,  6.390611 ,
        6.138003 ,  4.7280207,  5.6774774,  7.7608495,  5.5551353,
        6.7840095,  7.9837375,  7.886701 ,  7.5550613,  7.171591 ,
        8.072196 ,  5.616564 ,  9.523572 ,  7.669908 ,  4.8736596,
        9.889877 , 10.853143 , 11.421347 ,  6.1764994,  6.7230377,
        9.030946 ,  7.4831777,  4.9542456,  6.2279496,  5.5583076,
        4.7737303,  4.988839 ,  5.528941 ,  6.1055202,  4.9185286,
        1.4783832,  6.0847464,  6.2815223, 13.770041 ,  8.949925 ,
        4.632792 , 12.229613 ,  9.2779045,  7.643649 ,  5.6774344,
        8.359211 , 11.551817 ,  6.149157 ,  9.313957 ,  5.722327 ,
        6.135723 ,  6.8634076,  4.7212715,  5.687168 ,  7.1128035,
        5.2474313,  4.3592653,  2.9234428,  8.094502 ,  6.8743

In [82]:
# Calculating the accuracy
print("Accuracy:",round(xgb.score(X_train, Y_train)*100,2))

Accuracy: 99.91


In [83]:
submission = pd.DataFrame({
        "Index": test["Index"],
        "Price": y_pred
    })
submission.to_csv('https://raw.githubusercontent.com/MathewJohn1986/Team-8-Predict/main/sample_submission.csv', index=False)

TypeError: a bytes-like object is required, not 'str'

In [85]:
submission

,Index,Price
0,1,11.009044
1,2,11.942808
2,3,10.043292
3,4,8.694312
4,5,7.922956
...,...,...
680,681,3.284856
681,682,10.035151
682,683,6.110547
683,684,7.277902
